In [1]:
#import dependencies
import pandas as pd
import numpy as np
import warnings
from geopy.geocoders import Nominatim #for going from loctaion to coordinates (lan/lon)
from geopy.exc import GeocoderTimedOut

# Data Cleaning

In [2]:
#reading the csv file into a dataframe 
shootings_df = pd.read_csv('clean_shooting.csv', encoding='latin-1')
shootings_df.head()

,case,location,date,year,summary,fatalities,injured,total_victims,venue,prior_signs_mental_health,...,weapon_details,race,gender,sources,mental_health_sources,latitude,longitude,shooting_type,city,state
0,Yountville veterans home shooting,"Yountville, CA",3/9/18,2018,"Army veteran Albert Cheung Wong, 36, stormed a...",3,0,3,Workplace,Yes,...,NaN,Asian,Male,https://www.cnn.com/2018/03/10/us/california-v...,https://www.cnn.com/2018/03/10/us/california-v...,38.395445,-122.360173,Mass,Yountville,California
1,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,AR-15,White,Male,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,26.310079,-80.237270,Mass,Parkland,Florida
2,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,Unknown,...,NaN,White,Male,http://www.wpxi.com/news/top-stories/family-me...,NaN,40.052018,-79.389202,Mass,Melcroft,Pennsylvania
3,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,Unknown,...,Two illegally modified rifles,White,Male,https://www.nbcnews.com/news/us-news/californi...,NaN,39.993786,-122.471880,Spree,Rancho Tehama,California
4,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46,Religious,Yes,...,Ruger AR-556; Kelley also possessed semiautoma...,White,Male,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,29.274044,-98.055906,Mass,Sutherland Springs,Texas


In [ ]:
# #testing for Geopy to retrieve (latitude , longitude) based on the "Location" field of the dataframe 
# test_address = "Sutherland Springs, TX"
# geolocator=Nominatim(timeout=10)

# try:
#     location = geolocator.geocode(test_address)
#     print((location.latitude, location.longitude))
# except GeocoderTimedOut as e:
#     print("Error: geocode failed on input %s with message %s"%(test_address, e.message))

In [3]:
# adding redults to another dataframe so that there is no confusion when referencing for Database creation 
complete_shootings_df = shootings_df
complete_shootings_df.head()      

,case,location,date,year,summary,fatalities,injured,total_victims,venue,prior_signs_mental_health,...,weapon_details,race,gender,sources,mental_health_sources,latitude,longitude,shooting_type,city,state
0,Yountville veterans home shooting,"Yountville, CA",3/9/18,2018,"Army veteran Albert Cheung Wong, 36, stormed a...",3,0,3,Workplace,Yes,...,NaN,Asian,Male,https://www.cnn.com/2018/03/10/us/california-v...,https://www.cnn.com/2018/03/10/us/california-v...,38.395445,-122.360173,Mass,Yountville,California
1,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,AR-15,White,Male,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,26.310079,-80.237270,Mass,Parkland,Florida
2,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,Unknown,...,NaN,White,Male,http://www.wpxi.com/news/top-stories/family-me...,NaN,40.052018,-79.389202,Mass,Melcroft,Pennsylvania
3,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,Unknown,...,Two illegally modified rifles,White,Male,https://www.nbcnews.com/news/us-news/californi...,NaN,39.993786,-122.471880,Spree,Rancho Tehama,California
4,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46,Religious,Yes,...,Ruger AR-556; Kelley also possessed semiautoma...,White,Male,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,29.274044,-98.055906,Mass,Sutherland Springs,Texas


In [ ]:
# list of all column headers
list(complete_shootings_df)

In [ ]:
# #rename column headers so that there are no spaces & better usability

# complete_shootings_df = complete_shootings_df.rename(columns={'Case': 'case', 
#                                         'Location': 'location', 
#                                         'Date': 'date', 
#                                         'Year': 'year', 
#                                         'Summary': 'summary', 
#                                         'Fatalities': 'fatalities', 
#                                         'Injured': 'injured', 
#                                         'Total victims': 'total_victims', 
#                                         'Venue': 'venue', 
#                                         'Prior signs of mental health issues': 'prior_signs_mental_health', 
#                                         'Mental health - details': 'mental_health', 
#                                         'Weapons obtained legally': 'weapons_obtained_legally', 
#                                         'Where obtained': 'where_obtained',
#                                         'Type of weapons': 'type_weapon', 
#                                         'Weapon details': 'weapon_details', 
#                                         'Race': 'race', 
#                                         'Gender': 'gender', 
#                                         'Sources': 'sources', 
#                                         'Mental Health Sources': 'mental_health_sources', 
#                                         'Type': 'shooting_type'})

# complete_shootings_df.head()

In [ ]:
# # recheck column headers
# complete_shootings_df.info()

In [ ]:
# #update the new dataframe with city/state split from location
# complete_shootings_df[['city', 'state']] = complete_shootings_df.location.str.split(',', expand = True)
# complete_shootings_df.head()

In [4]:
list(complete_shootings_df)

['case',
 'location',
 'date',
 'year',
 'summary',
 'fatalities',
 'injured',
 'total_victims',
 'venue',
 'prior_signs_mental_health',
 'mental_health',
 'weapons_obtained_legally',
 'where_obtained',
 'type_weapon',
 'weapon_details',
 'race',
 'gender',
 'sources',
 'mental_health_sources',
 'latitude',
 'longitude',
 'shooting_type',
 'city',
 'state']

In [ ]:
# complete_shootings_df.state.value_counts()

In [ ]:
# # Condense all "White"
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('CA', 'California'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('PA', 'Pennsylvania'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('D.C.', 'Washington D.C.'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('TX', 'Texas'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('LA', 'Louisiana'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('WA', 'Washington'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('NV', 'Nevada'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('MD', 'Maryland'))
# complete_shootings_df.state = complete_shootings_df.state.apply(lambda x: x.replace('CO', 'Colorado'))

# # print
# complete_shootings_df.state.value_counts()


In [ ]:
# #clean up dataframe columns
# complete_shootings_df.race.value_counts()

In [5]:
# complete_shootings_df['race'].replace(['white', 'black', 'unclear'], 
#                                       ['White', 'Black', 'Unknown'], inplace=True)


# complete_shootings_df.race.value_counts()

White              56
Black              16
Asian               8
Latino              7
Other               5
Native American     3
Unknown             1
Name: race, dtype: int64

In [ ]:
# complete_shootings_df.gender.value_counts()

In [ ]:
# complete_shootings_df['gender'].replace(['M'], 
#                                       ['Male'], inplace=True)

# complete_shootings_df.gender.value_counts()

In [ ]:
# complete_shootings_df.venue.value_counts()

In [ ]:
# complete_shootings_df['venue'].replace(['Other\r', '\rWorkplace'], 
#                                       ['Other', 'Workplace'], inplace=True)

# complete_shootings_df.venue.value_counts()

In [ ]:
# complete_shootings_df.prior_signs_mental_health.value_counts()

In [ ]:
complete_shootings_df['prior_signs_mental_health'].replace(['TBD','Unclear'], 
                                      ['Unknown','Unknown'], inplace=True)


complete_shootings_df.prior_signs_mental_health.value_counts()

In [ ]:
complete_shootings_df.weapons_obtained_legally.value_counts()

In [ ]:

complete_shootings_df['weapons_obtained_legally'].replace(['TBD', '\rYes', 'Yes ("some of the weapons were purchased legally and some of them may not have been")', 'Kelley passed federal criminal background checks; the US Air Force failed to provide information on his criminal history to the FBI'], 
                                      ['Unknown','Yes','Yes','Unknown'], inplace=True)


complete_shootings_df.weapons_obtained_legally.value_counts()

In [ ]:
complete_shootings_df.total_victims.value_counts()

In [ ]:
complete_shootings_df['total_victims'].replace(['758+','11 (dozens more were reportedly injured in the panic)', '46+'], 
                                      ['758','11','46'], inplace=True)


complete_shootings_df.total_victims.value_counts()

In [ ]:
complete_shootings_df.injured.value_counts()

In [ ]:
complete_shootings_df['injured'].replace(['700+'], 
                                      ['700'], inplace=True)


complete_shootings_df.injured.value_counts()

In [ ]:
for col in complete_shootings_df.columns:
    try:
        str(complete_shootings_df[col].iloc[0])
        complete_shootings_df[col] = complete_shootings_df[col].apply(lambda x: x.replace('\r', ''))
    except:
        pass

In [ ]:
# Loop through the dataframe to check if the latitude is missing. 
# If the Latitue is a "NaN" then update the (Lat, Lon) data from Geopy
for index, row in complete_shootings_df.iterrows():
        try:
            location = geolocator.geocode(row['location'])
            complete_shootings_df.loc[index, 'latitude']= location.latitude
            complete_shootings_df.loc[index, 'longitude']= location.longitude
        except GeocoderTimedOut as e:
            print("Error: geocode failed on input %s with message %s"%(row['Location'], e.message))

In [ ]:
# adding redults to another dataframe so that there is no confusion when referencing for Database creation 
# Geopy San Francisco Coordinates are incorrect, so we'll update it manually 
complete_shootings_df.loc[83, 'longitude']= -122.41941550000001
complete_shootings_df.loc[83, 'latitude']= 37.7749295
complete_shootings_df.loc[8, 'longitude']= -122.41941550000001
complete_shootings_df.loc[8, 'latitude']= 37.7749295
complete_shootings_df[complete_shootings_df['location'] == 'San Francisco, CA']

In [ ]:
# complete_shootings_df.race.value_counts()
complete_shootings_df.head()

In [ ]:
complete_shootings_df.to_csv('clean_shooting.csv', index=False)

In [ ]:
pd.read_csv('clean_shooting.csv')

In [ ]:
complete_shootings_df.info()

# Database Creation

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import Column, Integer, String, Float

In [ ]:
Base = declarative_base()

In [ ]:
class Shootings(Base):
    __tablename__ = "shootings"
    
    id = Column(Integer, primary_key=True)
    case = Column(String)                        
    location = Column(String)                   
    date = Column(String)                         
    year = Column(Integer)                                 
    summary = Column(Integer)                                
    fatalities = Column(Integer)                             
    injured = Column(Integer)                               
    total_victims = Column(Integer)                         
    venue = Column(String)                                
    prior_signs_mental_health = Column(String)             
    mental_health = Column(String)                         
    weapons_obtained_legally = Column(String)           
    where_obtained = Column(String)                        
    type_weapon = Column(String)                         
    weapon_details = Column(String)                    
    race = Column(String)                               
    gender = Column(String)                              
    sources = Column(String)                            
    mental_health_sources = Column(String)             
    latitude = Column(Float)                            
    longitude = Column(Float)                              
    shooting_type = Column(String)                        
    city = Column(String)                                
    state = Column(String)                              
    

In [ ]:
# engine = create_engine("sqlite:///shootings.sqlite")
# Base.metadata.create_all(engine)
Base.metadata.tables

In [ ]:
engine = create_engine("sqlite:///shootings.sqlite")

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
# Create a session
session = Session(engine)

In [ ]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()

In [ ]:
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('shootings')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
complete_shootings_df.to_sql('shootings', engine, if_exists='append', index=False)

In [ ]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
session.commit()


In [ ]:
session.query(func.count(Shootings.case)).group_by(Shootings.year).all()

In [ ]:
data = session.query(func.count(Shootings.case).label('case_count'), Shootings.year, func.sum(Shootings.fatalities).label('fatalities') ).group_by(Shootings.year).all()
data

In [ ]:
data = session.query(func.count(Shootings.case).label('case_count'), Shootings.year, func.sum(Shootings.fatalities).label('fatalities') ).group_by(Shootings.year).all()
results = []
for row in data:
    data_dict = {}
    data_dict['year']= row.year
    data_dict['case_count'] = row.case_count
    data_dict['total_fatalities'] = row.fatalities
    results.append(data_dict)

In [ ]:
results